In [40]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
import gradio as gr

In [41]:
def load_env():
  load_dotenv()

def initialize_llm():
  llm = ChatGroq(
    temperature = 0,
    model_name = "llama-3.3-70b-versatile"
  )
  return llm

In [42]:
def create_vector_db():
  loader = DirectoryLoader(path="data", glob = '*.pdf', loader_cls = PyPDFLoader)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma.from_documents(texts, embeddings, persist_directory = './data/chroma_db')
  vector_db.persist()

  print("ChromaDB created and data saved")

  return vector_db

In [43]:
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])

In [ ]:
def setup_qa_chain(vector_db, llm, parser):
    retriever = vector_db.as_retriever()
    prompt_templates = """ You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
        {context}
        User: {question}
        Chatbot:
    """
    prompt  = PromptTemplate(template = prompt_templates, input_variables = ['context', 'question'])

    qa_chain = (
        {"context": retriever | format_docs,"question":RunnablePassthrough()}
        | prompt
        | llm
        | parser
    )

    return qa_chain

In [45]:
def main():
  print("Intializing Chatbot.........")
  load_env()
  llm = initialize_llm()

  db_path = "/data/chroma_db"
  parser = StrOutputParser()

  if not os.path.exists(db_path):
    vector_db  = create_vector_db()
  else:
    embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
  qa_chain = setup_qa_chain(vector_db, llm, parser)

  print("Chatbot is ready to talk! Type 'exit' to quit.")
  while True:
    query = input("\nHuman: ")
    if query.lower()  == "exit":
      print("Chatbot: Take Care of yourself, Goodbye!")
      break
    response = qa_chain.invoke(query)
    print(f"Chatbot: {response}")

if __name__ == "__main__":
  main()

Intializing Chatbot.........
ChromaDB created and data saved
Chatbot is ready to talk! Type 'exit' to quit.
Chatbot: Take Care of yourself, Goodbye!


In [ ]:
# using gradio to create a web interface for the chatbot
llm = initialize_llm()
load_env()
db_path = "/content/chroma_db"
parser = StrOutputParser()

if not os.path.exists(db_path):
  vector_db  = create_vector_db()
else:
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
qa_chain = setup_qa_chain(vector_db, llm, parser)

def chatbot_response(user_input,history):
  if not user_input.strip():
    return "Please provide a valid input"
  response = qa_chain.invoke(user_input)
  return response

gr.Markdown("# 🧠 Mental Health Chatbot 🤖")
gr.Markdown("A compassionate chatbot designed to assist with mental well-being. Please note: For serious concerns, contact a professional.")

chatbot = gr.ChatInterface(fn=chatbot_response, title="Mental Health Chatbot")

gr.Markdown("This chatbot provides general support. For urgent issues, seek help from licensed professionals.")

chatbot.launch()

Intializing Chatbot.........
ChromaDB created and data saved
* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
